<a href="https://colab.research.google.com/github/gyasifred/disease-prediction/blob/main/clinical_valence_testing_train_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
%ls | grep "fine-tune.py"

fine-tune.py


In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.6 MB/s eta 0:00:00


In [ ]:
!python fine-tune.py \
    --train_csv /content/drive/MyDrive/data/DIA_GROUPS_3_DIGITS_adm_train.csv  \
    --val_csv /content/drive/MyDrive/data/DIA_GROUPS_3_DIGITS_adm_val.csv \
    --num_epochs 20  \
    --batch_size 32  \
    --output_dir CORe \
    --threshold 0.5

Loading datasets...
Identifying all unique codes...
Found 1215 unique codes across datasets
Creating multi-hot encoded labels...
Training samples: 33994
Validation samples: 4918
Number of codes to predict: 1215
Using device: cuda
config.json: 100% 428/428 [00:00<00:00, 3.78MB/s]
vocab.txt: 100% 213k/213k [00:00<00:00, 4.78MB/s]
2025-03-09 05:22:10.635675: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-09 05:22:10.651716: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741497730.669866    2925 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been re

In [6]:
!python inference.py \
 --csv_path  /content/drive/MyDrive/data/DIA_GROUPS_3_DIGITS_adm_test.csv\
 --model_dir CORe/ \
 --threshold  0.3 --output_dir CORe_test_0.3_threshold --batch_size 32 --output_format csv --print_result

Using device: cuda
Loading model from CORe/...
2025-03-09 18:15:49.667569: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741544149.688510    5441 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741544149.694766    5441 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-09 18:15:49.716424: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some weights of BertForSequenceClassification were not initialized from the mo

In [7]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("bvanaken/CORe-clinical-outcome-biobert-v1")
model = AutoModel.from_pretrained("bvanaken/CORe-clinical-outcome-biobert-v1")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

In [8]:
text = """
CHIEF COMPLAINT: Positive ETT

PRESENT ILLNESS: Mr. [**Known lastname 1683**] is a pleasant 82 year old gentleman with known coronary artery disease, prior MI and PCI in the past. An ETT in [**2197-5-22**] depressions but negative for chest pain. Nuclear imaging showed a dilated LV with an ejection fraction of 24%. There was a large inferior and inferolateral fixed defect with a large reversible apical defect. Based upon the above results, he was referred for repeat cardiac catheterization. On admission, he denied chest pain, SOB, fatigue, syncope, palpitations and pedal edema. He reported one episode of dizziness which lasted only several seconds approximately one week prior to this admission.

MEDICAL HISTORY: Ischemic Cardiomyopathy, EF 24% CAD and AS History of MI and RCA stent [**2188**] Hyperlipidemia HTN BPH Prior Hernia repairs

MEDICATION ON ADMISSION: Zocor 40qd, Captopril 25 qd, Terazosin 5 qd, Aspirin 325 qd, MVI, Vit E, Vit C

ALLERGIES: Patient recorded as having No Known Allergies to Drugs

PHYSICAL EXAM: Vitals: BP 127/76, HR 75, RR 14, SAT  97%on room air General: well developed elderly male in no acute distress HEENT: oropharynx benign, Neck: supple, no JVD, Heart: regular rate, normal s1s2, 3/6 systolic murmur Lungs: clear bilaterally Abdomen: soft, nontender, normoactive bowel sounds Ext: warm, no edema, no varicosities Pulses: 2+ distally Neuro: alert and oriented, nonfocal

FAMILY HISTORY: Denies premature CAD.

SOCIAL HISTORY: Married with 3 children. He denies tobacco and excessive ETOH.
"""

In [20]:
import torch

# Determine device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model.to(device)
model.eval()

# Tokenize input (ensure `text` is defined)
inputs = tokenizer(
    text,
    truncation=True,
    padding=True,
    max_length=512,
    return_tensors="pt"
).to(device)

# Make prediction
with torch.no_grad():
    outputs = model(**inputs)

    # Check the output attributes
    print(outputs.keys())  # Debugging step to see available attributes

    if hasattr(outputs, "logits"):
        logits = outputs.logits
        probabilities = torch.sigmoid(logits).cpu().numpy()[0]
        predictions = (probabilities > 0.5).astype(int)
        print(predictions)
    elif hasattr(outputs, "last_hidden_state"):
        print("No logits found, using embeddings instead")
        embeddings = outputs.last_hidden_state  # Extract hidden states
        print(embeddings.shape)  # Debugging shape of the embeddings
    else:
        print("Unexpected model output structure:", outputs)

Using device: cuda
odict_keys(['last_hidden_state', 'pooler_output'])
No logits found, using embeddings instead
torch.Size([1, 432, 768])
